In [1]:
!pip install -qU sentence-transformers langchain langchain_milvus langchain-huggingface ollama langchain-ollama pypdf langchainhub   "pymilvus[model]"


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
poetry 1.4.2 requires requests-toolbelt<0.11.0,>=0.9.1, but you have requests-toolbelt 1.0.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
! ollama pull llama3.2


'ollama' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
import os
from pymilvus import MilvusClient
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_milvus import Milvus
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import hub

c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.2.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [5]:
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, list_collections

try:
    connections.connect(alias="default", host="127.0.0.1", port="19530")
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


In [7]:
vectorstore = Collection(name="Miri_Regev_quotes")

In [10]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_fn = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
vectorstore.load()

# Define a query vector (e.g., embedding for a new query)
query_vector = embedding_fn.encode(["מה דעתה תחבורה ציבורית?"])[0]

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = vectorstore.search(
    data=[query_vector],  # Query vector
    anns_field="vector",
    param=search_params,
    limit=15,  # Number of top matches
    output_fields=["text"],  # Include the text field in results
)

In [22]:
type(results)

pymilvus.client.abstract.SearchResult

In [ ]:
import json

# Extract text and distance from search results
retrieved_lines_with_distances = [
    (res.entity.get('text'), res.distance) for res in results[0]
]

# Print the results
print(retrieved_lines_with_distances)

[('לתחבורה הציבורית', 4.982583999633789), ('והתחבורה הציבורית ב', 7.834672451019287), ('קח תחבורה', 11.079657554626465), ('עכשיו התפקיד שלנו זה להוסיף את התדירות של האוטובוסים רכבות בעזרת השם  הרכבת מקריית שמונה ועד אילת מתקדמת  רכבות קלות להביא עוד ועוד כלים של תחבורה ציבורית כדי באמת לשכנע את האזרח לוותר על הרכב ולנסוע בתחבורה הציבורית', 11.761489868164062), ('ממשלה שאומרת שאכפת לה מהפריפריה ומהחלשים ובפועל מעלה ב את מחירי הנסיעה בתחבורה הציבורית', 12.796658515930176), ('יש כאן בעצם ראייה כוללת שמדברת על יכולת של הנגשה וזמינות לאזרח של תחבורה ציבורית  זמינה נגישה וכמובן גם ירוקה עד כמה שניתן', 12.96973991394043), ('אנחנו רוצים שבכל מתחמי התחנות אדם יגיע מהבית שלו ברכב דוגלגלי יחנה בתחנה ייסע ליעדו על רכבת או על אוטובוס חשמלי או אוטובוס אחר ויחזור', 13.739957809448242), ('משרד התחבורה הוא משרד גדול משרד שיש לו אחריות אזרחית על אוויר ים ויבשה על כל הכניסה והיציאה אל הארץ ומהארץ החוצה', 13.859049797058105), ('נדבר על הרכבת הקלה וגם על הכבדה', 14.734820365905762), ('אני רוצה לתת עוד בשור

In [32]:
context = "\nQuote:".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)


In [35]:
question = "מה דעתך על התחבורה הציבורית בישראל?"

In [33]:
context

'לתחבורה הציבורית\nQuote:והתחבורה הציבורית ב\nQuote:קח תחבורה\nQuote:עכשיו התפקיד שלנו זה להוסיף את התדירות של האוטובוסים רכבות בעזרת השם  הרכבת מקריית שמונה ועד אילת מתקדמת  רכבות קלות להביא עוד ועוד כלים של תחבורה ציבורית כדי באמת לשכנע את האזרח לוותר על הרכב ולנסוע בתחבורה הציבורית\nQuote:ממשלה שאומרת שאכפת לה מהפריפריה ומהחלשים ובפועל מעלה ב את מחירי הנסיעה בתחבורה הציבורית\nQuote:יש כאן בעצם ראייה כוללת שמדברת על יכולת של הנגשה וזמינות לאזרח של תחבורה ציבורית  זמינה נגישה וכמובן גם ירוקה עד כמה שניתן\nQuote:אנחנו רוצים שבכל מתחמי התחנות אדם יגיע מהבית שלו ברכב דוגלגלי יחנה בתחנה ייסע ליעדו על רכבת או על אוטובוס חשמלי או אוטובוס אחר ויחזור\nQuote:משרד התחבורה הוא משרד גדול משרד שיש לו אחריות אזרחית על אוויר ים ויבשה על כל הכניסה והיציאה אל הארץ ומהארץ החוצה\nQuote:נדבר על הרכבת הקלה וגם על הכבדה\nQuote:אני רוצה לתת עוד בשורה שהיא חשובה מאוד ולהודות כאן גם לחברי מקלב סגן השרה שהבאנו איגום משאבים ועצרנו את עליית המחירים של  בתחבורה הציבורית  עלייה שהיא באמת גזרה קשה על הציבור וזו עוד

In [36]:
SYSTEM_PROMPT = """
Human: אתה כרגע בוט שמייצג את מירי רגב במדינת ישראל, הנה ציטוטים שנאמרו על ידך 
"""
USER_PROMPT = f"""
Use the following Quotes enclosed in <context> tags to provide an answer to the question enclosed in <question> tags, answer like you were Miri Regev use the tone you find in the context quotes. answer in Hebrew.
<context>
{context}
</context>
<question>
{question}
</question>
"""


In [ ]:
# Load the model
import ollama

# The model is already pulled in CELL INDEX: 1, so you can directly use the chat function

AttributeError: module 'ollama' has no attribute 'load'

In [48]:
from ollama import chat
from ollama import ChatResponse

# Ensure the server is running and the correct host and port are being used
try:
    response: ChatResponse = chat(
        model="llama3.2",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": USER_PROMPT},
        ],
    )
    print(response["message"]["content"])
except Exception as e:
    print(f"An error occurred: {e}")


An error occurred: [WinError 10061] No connection could be made because the target machine actively refused it


In [39]:
from pymilvus import utility, Collection

# List all collections
collections = utility.list_collections()
print("Available collections:", collections)

# Access a specific collection
collection_name = "Miri_Regev_quotes"  # Replace with the name of your collection
collection = Collection(name=collection_name)

# Print schema
print("Schema:", collection.schema)


Available collections: ['Miri_Regev_quotes']
Schema: {'auto_id': False, 'description': 'Demo collection for quotes of Miri Regev', 'fields': [{'name': 'id', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'vector', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 384}}, {'name': 'text', 'description': '', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 512}}], 'enable_dynamic_field': False}
